In [ ]:
# |default_exp analysis_utility
# You need this at the top of every notebook you want turned into a module, the name your provide will determine the module name

# Libraries


In [ ]:
# |export
# That export there, it makes sure this code goes into the module.

# standard libs
import os
import re

# Common to template
# add into settings.ini, requirements, package name is python-dotenv, for conda build ensure `conda config --add channels conda-forge`
import dotenv  # for loading config from .env files, https://pypi.org/project/python-dotenv/
import envyaml  # Allows to loads env vars into a yaml file, https://github.com/thesimj/envyaml
import fastcore  # To add functionality related to nbdev development, https://github.com/fastai/fastcore/
from fastcore import (
    test,
)
from fastcore.script import (
    call_parse,
)  # for @call_parse, https://fastcore.fast.ai/script
import json  # for nicely printing json and yaml
from fastcore import test
from fastcore.script import call_parse
from ssi_analysis_utility import (
    core,
    sample_manager,
    convert_external_genome,
    
)
from pathlib import Path #to be able write :Path in cli function

# Project specific libraries
import subprocess
import pandas as pd
import logging
import csv

In [ ]:
# To change working directory when running notebook only. DO NOT EXPORT

os.chdir(core.PROJECT_DIR)

#### Analysis utility

The generic analysis_utility class inherits variables from the sample_data class: sample_name, file paths (assembly, Illumina, Nanopore), metadata etc.
One extra attribute is added in the initiation of analysis_utility: an output folder to store the analysis results ("analysis_utility.analysis_folder)

It contains methods for generic analysis and functionality that are used across species such as:

- Blasting genes and parsing output for presence absence of those genes
- Generating vcf file from Illumina reads (running bwa-mem and samtools against provided reference, can be used for both genes and whole genome sequences. The vcf output is formatted to be compatible with NASP)
- Parsing vcf file to identify variants at specified positions (for lineage determination and identification of specific SNPs in genes)
- Generating frankenfasta file from assembly (Aligning assembly to provided reference using NUCMER. Frankenfasta output can be concatenated with frankenfastas from other isolates to obtain a whole genome alignment if they use the same reference)
- Parsing frankenfasta file to identify variants at specified positions (for lineage determination and identification of specific SNPs in genes)


## Standards

Each analysis has a defined alias, f.ex. "virulence_gene_detection", "emm_typing" or "lineage_determination". This alias is provided as a parameter when calling the method that does the analysis, and results and output files for each analysis are stored in dicts as parameters .analysis_results and .analysis_output_files

.analysis_results should be a dictionary of dictionaries with strings as values. The top level key should be the analysis alias, and the keys in the second tier will be the tsv-headers in the final summary output, while the values will be printed to the field under that header.

Like this:

    analysis_alias_1:
        Test1: Result 1
        Test2: Result 2
    analysis_alias_2:
        Test1: Result 1
    analysis_alias_3:
        Test1: Result 1
        ....


For example if the .analysis_result attribute looks like this after all analyses are run

    "emm_typing":
        "emm_type": "4.0"
        "enn_type": "203.3"
        ...
    "lineage_determination":
        "Lineage": "M4 cluster 2"
    "virulence_gene_detection:
        "speA": 0
        "speB": 1
        "speC": 1


The summary output tsv will look like this:


sample_name | assembly_file   |    ....... | emm_type | enn_type  | ... | Lineage     |  speA | speB | speC
:---- | :----- | :----- | :----- | :----- | :----- | :----- | :----- | :----- | :-----
GAS-2024-0773 | GAS-2024-0773.fasta | .......|  4.0    |   203.3  | ... | M4 cluster 2 | 0   |  1  |   1


By default, files generated for analysis are printed to /provided/output/path/[analysis_alias]

# An analysis may use multiple methods
F.ex. lineage_determination consists of three steps;
1. mapping and variant calling based on reads or assembly data
    - Using methods ._Illumina_read_mapping or ._nucmer_mapping
2. parsing the vcf or frankenfasta output for variant positions matching those provided in a specific file
    - Using methods .get_SNPs_from_vcf or .get_SNPs_from_fasta
3. converting that information to a lineage.

.\_assembly_lineage_determination\_ or ._Illumina_lineage_determination\_ performs all three of the above and therefore the whole analysis in one method

Methods that start with "\_" use input contained in the attributes as input, i.e. assembly file or read files. It may also require other input, like a reference file to map against, but it can be used as a first step in an anlysis. If a method does not beging with "\_", that means it needs the output from a different method as input.

Methods that end in "_" are final steps of an analysis, i.e. these will update the dictionary in analysis_utility.analysis_results[analysis_alias] with results from the 
it's output needs to be passed on to another method to get results. These do not add anything to the .analysis_results dictionary, but may add to the .analysis_output_files dictionary. It should also return what is relevant for downstream analysis (f.ex. a path to a vcf-file, or one or more dictinaries and/or dataframes).

## Other notes:

### Naming requirements
Do _not_ use commas (,), colons (:) and spaces in your analysis_alias names


### Keeping intermediate files

The config for every analysis should have a "files_to_clean" key with a list of files that are deleted after analysis is run. Simply edit that list in the config before calling the function to only delete a subset of files





In [ ]:
# |export

class analysis_utility(sample_manager.sample_data):
    """
    Analysis_utility class, which extends sample_manager.sample_data
    """

    # Define the log file name for logging analysis processes
    log_file_name = "analysis.log"

    # Initialize the analysis_utility class with provided attributes and settings
    def __init__(self,
                 attributes,
                 input_folder,
                 output_folder,
                 analysis_config):
        """
        Initialize the analysis utility class by setting up attributes and configurations.

        Parameters:
        -------------
        attributes : dict
            The attributes related to the sample data.
        input_folder : str
            Path to the folder where input files are located.
        output_folder : str
            Path where the analysis results will be saved.
        analysis_config : dict
            Configuration settings for the analysis, including analysis type and parameters.
        """

        attributes = attributes.copy() # Make a copy of attributes to avoid modifying original data
        super().__init__(attributes,
                         input_folder) # Call the parent class initializer
        self.analysis_results = {} #  Initialize an empty dictionary for storing analysis results
        self.analysis_output_files = {} ## Initialize an empty dictionary for storing analysis output files
        self.output_folder = output_folder ## Set the output folder for saving results
        self.analysis_config = analysis_config # Store the analysis configuration
        self.check_path_existence() # Check the existence of paths provided in the configuration

    # Setup up log filehandle for sample
    def setup_sample_logging(self):
        """
        Method to initialize logging file for the sample to track the analysis process.
        """

        self.logger = logging.getLogger() # Get the logger instance
        log_file = os.path.join(self.output_folder,
                                self.log_file_name) # Define the log file path
        self.log_filehandle = logging.FileHandler(log_file, 
                                                  mode = "w",
                                                  encoding = "utf-8") # Create the log file handler
        self.log_filehandle.setFormatter(logging.Formatter(
            fmt="{asctime} - {levelname} - {message}",
            style="{",
            datefmt="%Y-%m-%d %H:%M:%S",
            )) # Log and date format 
        self.logger.addHandler(self.log_filehandle)
        self.logger.setLevel(logging.INFO) # Set the logging level to INFO

    def check_path_existence(self):
        """
        Method to verify the existence of the provided input files (assembly, Illumina, Nanopore).
        If a file does not exist, it is set to None and a warning is logged.
        """

        # Check if the assembly file exists
        if not self.assembly_file is None and not os.path.exists(self.assembly_file):
            self.logger.warning(f"Provided assembly file does not exist at {self.assembly_file}. Analyses dependent on assembly input will be skipped")
            self.assembly_file = None
        # Check if the Illumina read files exist (R1 and R2)
        if not self.Illumina_read_files is None and not os.path.exists(self.Illumina_read_files[0]):
            self.logger.warning(f"Provided paired end R1 file does not exist at {self.Illumina_read_files[0]}. Analyses dependent on paired end input will be skipped")
            self.Illumina_read_files = None
        if not self.Illumina_read_files is None and not os.path.exists(self.Illumina_read_files[1]):
            self.logger.warning(f"Provided paired end R2 file does not exist at {self.Illumina_read_files[1]}. Analyses dependent on paired end input will be skipped")
            self.Illumina_read_files = None
        # Check if the Nanopore read file exists
        if not self.Nanopore_read_file is None and not os.path.exists(self.Nanopore_read_file):
            self.logger.warning(f"Provided Nanopore data file does not exist at {self.Nanopore_read_file}. Analyses dependent on Nanopore input will be skipped")
            self.Nanopore_read_file = None


    # Set up folder to store results for specific analysis. 
    def analysis_setup(self,
                       analysis_alias,
                       output_folder= False):
        """
        Sets up a folder for saving the results of a specific analysis.
        Default path: [output_folder]/[sample_name]/[analysis_alias]

        Parameters:
        -------------
        analysis_alias : str
            The alias/name of the analysis being run.
        output_folder : str, optional
            The folder path for saving the results. If False, a default path will be used.
        
        Returns:
        --------
        str : The output folder path where results will be saved.
        """
        
        # Log the analysis being run
        self.logger.info(f"------------------------------------------------------------------------------------------")
        self.logger.info(f"RUNNING ANALYSIS: {analysis_alias}")
        if not output_folder:
            output_folder = os.path.join(self.output_folder,analysis_alias) # Set the default output folder
        if not os.path.exists(output_folder):
            os.makedirs(output_folder) # Create the output folder if it doesn't exist
        if analysis_alias in self.analysis_results:
            self.logger.warning(f"An analysis with name {analysis_alias} has already been performed. Outputs in {output_folder} may be overwritten.")
        self.logger.info(f"Printing {analysis_alias} results for {self.sample_name} to {output_folder}")  # Log the output location
        self.analysis_results[analysis_alias] = {} ## Initialize an empty dictionary for the analysis results
        self.analysis_output_files[analysis_alias] = {} ## Initialize an empty dictionary for output files
        return(output_folder)

    
    def sample_setup(self):
        """
        Method to initialize the folder for the sample and sets up logging for the analysis.
        """
        if not os.path.exists(self.output_folder):
            try:
                os.makedirs(self.output_folder) # Create the output folder for the sample if it doesn't exist
            except Exception as e:
                print(f"Failed to create folder for {self.sample_name} in {self.output_folder}. {e}")
                #self.logger.critical(f"Failed to create folder for {self.sample_name} in {self.output_folder}. {e}")
        self.setup_sample_logging() # Set up the logging for the sample
        self.logger.info(f"Running analyses on {self.sample_name} in {self.output_folder}")
        self.logger.info(f"Inputs:")  # Log input data
        if not self.assembly_file is None:
            self.logger.info(f"Assembly file: {self.assembly_file}")
        if not self.Illumina_read_files is None:
            self.logger.info(f"Illumina read files: {', '.join(self.Illumina_read_files)}")
        if not self.Nanopore_read_file is None:
            self.logger.info(f"Nanopore read file: {self.Nanopore_read_file}")
        self.logger.info(f"------------------------------------------------------------------------------------------")
        self.logger.info(f"Analyses to run: {', '.join(self.analysis_config['analyses_to_run'])}")

 
    def sample_cleanup(self):
        """
        Method to clean up logging resources by removing the log file handler.
        """
        self.logger.removeHandler(self.log_filehandle) # Remove the file handler
        self.log_filehandle.close() # Close the log file

 

    def analysis_cleanup(self,
                         analysis_alias,
                         files_to_clean):
        """
        Cleans up intermediate output files generated during an analysis.
        To be run as the last step of any analysis after updating self.analysis_results

        Parameters:
        -------------
        analysis_alias : str
            The alias of the analysis whose temporary files need to be cleaned.
        files_to_clean : list
            A list of file names to be cleaned up after analysis.
        """
        
        self.logger.info(f"Cleaning temporary files from analysis {analysis_alias}")
        output_files = self.analysis_output_files[analysis_alias] # Get the output files dictionary for the analysis
        output_file_folders = set() # Set to keep track of output file directories
        
        # Iterate over files to clean and delete them
        for file_to_clean in files_to_clean:
            if os.path.exists(output_files[file_to_clean]):
                self.logger.info(f"Removing {output_files[file_to_clean]}") # Log the file removal
                os.remove(output_files[file_to_clean]) # Remove the file
            output_file_folders.add(os.path.dirname(output_files[file_to_clean]))  # Add the directory to the set
        
        # Remove empty directories after cleaning files
        for path in output_file_folders:  
            if os.path.exists(path) and not os.path.isfile(path) and not os.listdir(path):
                try:    
                    self.logger.info(f"Cleaned all files from {path}. Removing empty directory")
                    os.rmdir(path) # Remove the empty directory
                except Exception as e:
                    self.logger.warning(f"Attempt to remove directory {path} failed. {e}")

 
    def write_to_tsv(self,
                     include_metadata = True):
        """
        Write results from all analyses to .tsv file. Includes metadata by default.

        Parameters:
        -------------

        include_metadata : bool
            Wether to include metadata
        """
        
        output_file = os.path.join(self.output_folder,
                                   "results_summary.tsv") # Where to store the file
        if include_metadata:
            print_dict = self.metadata.copy() # Include metadata
            # If 'Illumina_read_files' is a list, convert it to a comma-separated string
            if 'Illumina_read_files' in print_dict and isinstance(print_dict['Illumina_read_files'], list):
                print_dict["Illumina_read_files"] = ",".join(print_dict["Illumina_read_files"])
             # Move 'sample_name' to the first column for readability
            if "sample_name" in print_dict:
                print_dict = {'sample_name': print_dict.pop('sample_name'), **print_dict} 
        else:
            # Only include 'sample_name' in the output if metadata is excluded
            if "sample_name" in print_dict:
                print_dict = {"sample_name": self.sample_name}
            else:
                print_dict = {}
        
        # Add analysis results to the dictionary to be printed
        for analysis_alias in self.analysis_results:
            print_dict.update(self.analysis_results[analysis_alias])
        
        # Write the results to the output file, catching any exceptions
        try:
            with open(output_file, 'w') as o:
                dict_writer = csv.DictWriter(o, fieldnames=list(print_dict.keys()), delimiter='\t')
                dict_writer.writeheader()
                dict_writer.writerows([print_dict])  
            o.close()
            self.logger.info(f"Analysis summary written to {output_file}")
        except Exception as e:
            self.logger.critical(f"ERROR while writing analysis summary to {output_file}: {e}")


   
    def execute_cmd_and_log(self,
                            cmd,
                            log_stdout = True):
        """
        Method to execute a shell command, logging stdout and stderr automatically.
        
        Parameters:
        -------------
        cmd : str
            The command to be executed.
        log_stdout : bool
            Whether to log stdout.
        
        Returns:
        -------------
        stdout : str
            Standard output from the command.
        stderr : str
            Standard error from the command.
    """     
        
        # Run the shell command and capture output
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True, encoding='utf-8')
        stdout,stderr = process.communicate()
        
        # Log the command and its output
        self.logger.info(f"Running command: {cmd}")
        if log_stdout and stdout and stdout is not None:
            self.logger.info(f"Shell command STDOUT: {stdout}")
        if stderr and stderr is not None:
            self.logger.error(f"Shell command STDERR: {stderr}")
        return(stdout,stderr)


    ### Analysis functions
    
    ## Below are functions involved in running individual analyses

    # 
    # 
    # VCF output can be used directly in nasp
    def _Illumina_read_mapping(self,
                               analysis_alias,
                               reference_fasta_file, 
                               output_folder = False):
        """
        Method to map paired-end Illumina reads to a reference genome 
        using BWA MEM, then perform variant calling with GATK.
        VCF output can be used for lineage determination when mapping 
        to complete genome and for checking for SNPs in specific genes 
        (._get_SNPs_from_vcf_)
        Map paired end Illumina reads to provided reference with bwa mem 
        and do variant calling with gatk. Returns path to vcf-file.
        
        Parameters:
        -------------
        analysis_alias : str
            Alias for this analysis.
        reference_fasta_file : str
            Path to the reference FASTA file.
        output_folder : str
            Folder to store output files. Default is the main output folder.
        
        Returns:
        -------------
        vcf_file : str
            Path to the generated VCF file.
        """

        # Set output folder and file paths
        if not output_folder:
            output_folder = os.path.join(self.output_folder,analysis_alias)
        bam_prefix = os.path.join(output_folder,self.sample_name+"-bwamem")
        bam_file = bam_prefix+".bam"
        vcf_file = os.path.join(output_folder,self.sample_name+".vcf")
        
        # Define commands for BWA, Samtools, and GATK
        bwamem_cmd = f"bwa mem -R \'@RG\\tID:{self.sample_name}\\tSM:{self.sample_name}\'  -t 4 {reference_fasta_file} {self.Illumina_read_files[0]} {self.Illumina_read_files[1]} | samtools view -S -b -h - | samtools sort -o {bam_file}"
        samtoolsindex_cmd =  f"samtools index {bam_file}"
        gatk_cmd = f"java -Xmx10G -jar GenomeAnalysisTK.jar -T UnifiedGenotyper -dt NONE -glm BOTH -I {bam_file} -R {reference_fasta_file} -nt 4 -o {vcf_file} -out_mode EMIT_ALL_CONFIDENT_SITES -baq RECALCULATE -stand_call_conf 100 -ploidy 1"
        commands = {}
        
        # Execute the appropriate commands based on the existing files
        if os.path.exists(vcf_file):
            self.logger.info(f"vcf file found at {vcf_file}. Skipping read mapping")
        else:
            if os.path.exists(bam_file+".bai"):
                cmd = gatk_cmd
                self.logger.info(f"Indexed bam file found at {bam_file}.bai. Running gatk.")
            else:
                if os.path.exists(bam_file):
                    cmd = f"{samtoolsindex_cmd}; {gatk_cmd}"
                    self.logger.info(f"Bam file found at {bam_file}. Indexing and running gatk.")
                else:
                    cmd = f"{bwamem_cmd}; {samtoolsindex_cmd}; {gatk_cmd}"
                    self.logger.info(f"Running bwamem and gatk")
            
            # Run the constructed command
            process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True, encoding='utf-8')
            commands[cmd] = process.communicate()
            
            # Save output files to dictionary
            self.analysis_output_files[analysis_alias] = {"vcf": vcf_file, "bam": bam_file, "bam_index": bam_file+".bai"}
        return(vcf_file)


    def get_SNPs_from_vcf(self,
                          analysis_alias,
                          genotype_variant_table,
                          vcf_file):
        """
        Parse a VCF file and a genotype variant table (from ._Illumina_read_mapping) to identify  
        variants matching those listed in the genotype_variant_table 

        Parameters:
        -------------
        analysis_alias : str
            Alias for the analysis.
        genotype_variant_table : str
            Path to the genotype variant table file.
        vcf_file : str
            Path to the VCF file.
        
        Returns:
        -------------
        LOC_df : pd.DataFrame
            DataFrame containing all variants listed in the genotype variant table.
        vcf_df : pd.DataFrame
            DataFrame containing the complete VCF file.
        LOC_df_filtered : pd.DataFrame
            DataFrame containing variants found in both the variant table and VCF file.
        """       
       
        # Load the VCF and variant table files into dataframes
        vcf_df = pd.read_csv(vcf_file, sep='\t', header = None,comment="#")
        vcf_df.columns = ["referenceID","position","ID","ref","alt","qual","filter","info","format","count"]
        LOC_df = pd.read_csv(genotype_variant_table, sep='\t')
         
        # Create a unique variant identifier for matching between the two files
        LOC_df["variant"] = LOC_df["referenceID"] + "::" + LOC_df["ref"] + LOC_df["position"].astype(str) + LOC_df["alt"]
        vcf_df["variant"] = vcf_df["referenceID"] + "::" + vcf_df["ref"] + vcf_df["position"].astype(str) + vcf_df["alt"]
        
         # Filter the VCF DataFrame based on the variants in the variant table
        variants_filter_vals = LOC_df["variant"]
        vcf_df_filtered = vcf_df.query('variant in @variants_filter_vals')
        
        # Create a DataFrame for variants found in both files
        variants_found = list(vcf_df_filtered["variant"])
        LOC_df_filtered = LOC_df.query('variant in @variants_found')

        # Calculate SNP counts for each genotype
        genotype_SNP_counts = {}
        for genotype in set(LOC_df["genotype"]):
            try:
                print(f"{genotype} {LOC_df_filtered['genotype'].value_counts()[genotype]} {LOC_df['genotype'].value_counts()[genotype]}")
                genotype_SNP_counts[genotype] = {}
            except KeyError:
                print(f"{genotype} 0 {LOC_df['genotype'].value_counts()[genotype]}")
        return(LOC_df,vcf_df,LOC_df_filtered)

    
    def _nucmer_mapping(self,
                        analysis_alias,
                        reference_fasta_file,
                        output_folder = False):
        """
        Maps the assembly sequences to a provided reference using Nucmer, producing a "frankenfasta" file.
        The frankenfasta output is aligned to the reference and can be used for lineage determination
        and SNP detection within specific genes (._get_SNPs_from_frankenfasta).

        Parameters:
        -------------
            analysis_alias : str
                    Identifier for the analysis, used to label output files.
            reference_fasta_file : str
                    Path to the reference fasta file.
            output_folder : str 
                    Directory where output files are saved. Defaults to a subfolder in self.output_folder.

        Returns:
            str: Path to the generated frankenfasta file.
        """
        
        # Set the output folder based on provided path or default to a folder named after the analysis alias
        if not output_folder:
            output_folder = os.path.join(self.output_folder,analysis_alias)
        
        # Initialize an external genome object and import the assembly fasta file into it
        external_genome = convert_external_genome.Genome()
        external_genome.import_fasta_file(self.assembly_file)
        
        # Define file paths based on sample name and output folder
        prefix = os.path.join(output_folder,self.sample_name)
        delta_file = f"{prefix}.delta"
        filtered_delta_file = f"{prefix}.filtered.delta"
        frankenfasta_file = f"{prefix}.frankenfasta"
        
        # Check if frankenfasta file already exists to avoid re-running the process
        if not os.path.exists(frankenfasta_file):
            self.logger.info(f"Running nucmer and writing frankenfasta file to {frankenfasta_file}")
            
            # Command to run Nucmer with the reference and assembly files
            delta_cmd = f"nucmer --prefix={prefix} {reference_fasta_file} {self.assembly_file}"
            # Execute the delta command and log any errors, then execute the delta-filter command
            delta_filter_cmd = f"delta-filter -q -r -o 100 {delta_file} > {filtered_delta_file}"
            
            # Execute the delta command and log any errors, then execute the delta-filter command
            stdout,stderr = self.execute_cmd_and_log(delta_cmd,
                                                     log_stdout=False)
            stdout,stderr = self.execute_cmd_and_log(delta_filter_cmd)
            
            # Re-initialize genome objects to handle frankenfasta generation
            external_genome = convert_external_genome.Genome()
            external_genome.import_fasta_file(self.assembly_file)
            franken_genome = convert_external_genome.Genome()
            
            # Parse the filtered delta file to create the frankenfasta genome based on the mapping
            convert_external_genome.parse_delta_file(( filtered_delta_file ), 
                                                     franken_genome, 
                                                     external_genome)
             # Write the frankenfasta genome to a fasta file, adding sample name and reference to the header
            franken_genome.write_to_fasta_file(( frankenfasta_file ), self.sample_name + " ref:")
        else:
            self.logger.info(f"Frankenfasta file already exists at {frankenfasta_file}, skipping nucmer alignment")
        # Store the paths of the generated output files in analysis_output_files for reference
        self.analysis_output_files[analysis_alias].update({"frankenfasta": frankenfasta_file, 
                                                           "delta": delta_file, 
                                                           "filtered_delta": filtered_delta_file})
        return(frankenfasta_file)


    def get_SNPs_from_fasta(self,
                            variant_table_file,
                            mapped_fasta_file):
        """
        Extracts SNPs from a mapped FASTA file by comparing against a variant table.
        
        Parameters:
        -------------

        variant_table_file : str 
                Path to the variant table file in TSV format.
        mapped_fasta_file : str
                Path to the mapped FASTA file.

        Returns:
            tuple: DataFrames with all variants and filtered variants based on presence in the FASTA sequence.
        """

        fasta_dict = {} # Dictionary to store sequences from the FASTA file

        # Read mapped FASTA file and store sequences in fasta_dict
        with open(mapped_fasta_file) as f:
            for line in f:
                line = line.rstrip('\n')
                if line[0] == ">":
                    header = line.split("ref:")[1]
                    fasta_dict[header] = ""
                else:
                    fasta_dict[header]+= line
        
        # Load the variant table and create a variant column with unique SNP identifier
        LOC_df = pd.read_csv(variant_table_file, sep='\t')
        LOC_df["variant"] = LOC_df["referenceID"] + "::" + LOC_df["ref"] + LOC_df["position"].astype(str) + LOC_df["alt"]

        LOC_df_dict = LOC_df.to_dict("index")
        variants_filter_vals = LOC_df["variant"]
        variants_found = []
        
        # Check each variant to see if it matches the corresponding sequence in the FASTA file
        for idx in LOC_df_dict:
            if LOC_df_dict[idx]["referenceID"] in fasta_dict:
                if fasta_dict[LOC_df_dict[idx]["referenceID"]][(LOC_df_dict[idx]["position"]-1)] == LOC_df_dict[idx]["alt"]:
                    variants_found.append(LOC_df_dict[idx]["variant"])

        LOC_df_filtered = LOC_df.query('variant in @variants_found') # Filter for found variants

        return(LOC_df,LOC_df_filtered)

    
    def _assembly_lineage_determination_(self,
                                         lineage_determination_config,
                                         mapped_fasta_file = False,
                                         output_folder = False):
        """
        Determines the lineage of an assembly by comparing SNP presence against a lineage variant file.

        Parameters:
        -------------
        lineage_determination_config : dict
                Configuration for lineage determination.
        mapped_fasta_file : str 
                Path to mapped FASTA file.
        output_folder : str
                Path to save outputs.

        """    
        
        analysis_alias = lineage_determination_config["alias"]
        
        # Skip analysis without assembly file
        if self.assembly_file is None:
            self.logger.critical(f"Assembly file not provided or not found. Skipping analysis {analysis_alias}")
            self.analysis_results[analysis_alias] = {analysis_alias: "NA"}
        else:
            output_folder = self.analysis_setup(analysis_alias,
                                                output_folder)
            reference_fasta_file = lineage_determination_config["reference_fasta_file"]
            variant_table_file = lineage_determination_config["lineage_variant_file"]
            
            # If no mapped file provided, generate one
            if not mapped_fasta_file or not os.path.exists(mapped_fasta_file):
                mapped_fasta_file = self._nucmer_mapping(analysis_alias,
                                                         reference_fasta_file)
            
            # Get all and found SNPs
            all_variants, found_variants = self.get_SNPs_from_fasta(variant_table_file,
                                                                   mapped_fasta_file)
            lineage_SNP_counts = {}
            
            # Calculate lineage SNP percentages
            for genotype in set(all_variants["genotype"]):
                try:
                    lineage_SNP_counts[genotype] = found_variants['genotype'].value_counts()[genotype] / all_variants['genotype'].value_counts()[genotype] * 100
                except KeyError:
                    lineage_SNP_counts[genotype] = 0
            
            # Determine the best lineage hit based on percentage threshold
            best_hit = max(lineage_SNP_counts, key=lineage_SNP_counts.get)
            if not lineage_SNP_counts[best_hit] > lineage_determination_config["percent_snp_threshold"]:
                best_hit = "-"
            
            self.analysis_results[analysis_alias].update({"Lineage": best_hit})
            self.analysis_cleanup(analysis_alias,lineage_determination_config["files_to_clean"])



    def _blast_presence_absence_(self,
                                 blast_presence_absence_config: dict,
                                 output_folder = False):
        """
        Executes BLAST for presence-absence analysis of genes in a query FASTA file.
        A gene is considered present if above the threshold specified in config 
        (default: 90 % identity, 90% length)

        Parameters:
        -------------
        blast_presence_absence_config : dict
                Configuration for BLAST parameters.
        output_folder : str
                Path to save BLAST output.

        Returns:
            dict: Presence-absence data and genes found.
        """

        analysis_alias = blast_presence_absence_config["alias"]
        
        # Skip analysis without assembly file
        if self.assembly_file is None:
            self.logger.critical(f"Assembly file not provided or not found. Skipping analysis {analysis_alias}")
            self.analysis_results[analysis_alias] = {analysis_alias: "NA"}
        else:
            # Set up options
            output_folder = self.analysis_setup(analysis_alias,output_folder)
            query_sequence_file = blast_presence_absence_config["query_fasta_file"]
            blast_output_file = os.path.join(output_folder,
                                             "blast_output.tsv")
            cov_threshold = blast_presence_absence_config["cov_threshold"]
            pident_threshold = blast_presence_absence_config["pident_threshold"]
            gene_presence_absence = {} # Create presence-absence dictionary for each gene
            genes_found = []
            gene_names = get_names_from_fasta(query_sequence_file)
            
            # Run BLAST if output file does not exist
            if not os.path.exists(blast_output_file):
                cmd = f"blastn -query {query_sequence_file} -subject {self.assembly_file} -out {blast_output_file} -outfmt \"6 {blast_presence_absence_config['blast_header']}\" {blast_presence_absence_config['additional_blast_parameters']}"
                stdout,stderr = self.execute_cmd_and_log(cmd)
            else:
                self.logger.info(f"Blast output found at {blast_output_file}, skipping blast.")
            
            # Parse BLAST output
            self.logger.info(f"Parsing blast output in {blast_output_file}")
            try:
                blast_df = pd.read_csv(blast_output_file, 
                                       sep='\t', 
                                       header = None)
            except Exception as e:
                self.logger.info(f"Failed to parse blast output in {blast_output_file}, error message: {e}")
                blast_df = None 
            if blast_df is not None:
                blast_df.columns = blast_presence_absence_config["blast_header"].split(' ')
                blast_df["plen"] = blast_df["length"]/blast_df["qlen"]*100
                blast_df_filtered = blast_df.query("plen > @cov_threshold and pident > @pident_threshold")
                blast_df_unique = blast_df_filtered.sort_values(by=['bitscore'], ascending= False).groupby("qseqid").first()
                genes_found =  ",".join(sorted(list(blast_df_unique.index)))
                genes_present = {f"{analysis_alias}_genes_found": genes_found}       
            else:
                blast_df_unique = None
            
            ##### Not ideal code ####
            
            # Create presence-absence dictionary for each gene
            for gene_name in gene_names:
                if gene_name in genes_found:
                    gene_presence_absence[gene_name] = "1"
                else:
                    gene_presence_absence[gene_name] = "0"
            
            # Update analysis results
            if blast_presence_absence_config["results_format"] == "string":
                self.analysis_results[analysis_alias] = genes_present
            else:
                self.analysis_results[analysis_alias] = gene_presence_absence
            self.analysis_output_files[analysis_alias] = {"blast": blast_output_file}
            self.analysis_cleanup(analysis_alias,blast_presence_absence_config["files_to_clean"])
            
            return({"genes_present": genes_found,
                    "gene_presence_absence": gene_presence_absence,
                    #"blast_df": blast_df_unique
                    })

    def __iter__(self):
        for analysis_alias in self.analysis_results:
            yield(analysis_alias)

    def __getitem__(self, analysis_alias):
        return({"results": self.analysis_results[analysis_alias],
                "output_files": self.analysis_output_files[analysis_alias]})

  


In [ ]:
# |export

class analysis_manager(sample_manager.input_manager):
    """
    Analysis manager class that inherits from `input_manager`
    """
    
    def __init__(self,
                 input_config,
                 analysis_settings_config):
        """
        Initialize the analysis utility class by setting up attributes and configurations.

        Parameters:
        -------------
        input_config : yaml file
            Configuration file with options to handle samples
        analysis_settings_config : dict
            Dictionary with analysis to run
        """
        
        # Initialize the analysis settings configuration
        self.analysis_settings_config: dict = analysis_settings_config
        
        # Set the base output folder; if not defined in input config, set it to the current directory
        if not "output_folder" in input_config:
            input_config["output_folder"] = "./"
        
        # Get absolute path for the base output folder
        self.base_output_folder = os.path.abspath(input_config["output_folder"])
        
        if not os.path.exists(self.base_output_folder):
            os.makedirs(self.base_output_folder)  # Create the output folder if it doesn't exist
        
        # Initialize the base class (input_manager) with input configuration
        super().__init__(input_config)
        



    def init_sample(self,
                    attributes):
        """
        Initialize analysis_utility and add class instance to list in analysis_manager.samples

        Parameters
        -------------
        attributes : dict 
            dictionary containing sample information 
        """
        # Determine the output folder for the sample

        if "output_folder" in attributes:
            output_folder = os.path.abspath(attributes["output_folder"])
        elif "sample_name" in attributes:
            output_folder = os.path.join(self.base_output_folder,attributes["sample_name"])
        else:
            output_folder = self.base_output_folder
        # Create an instance of `analysis_utility` with the appropriate attributes
            
        sample = analysis_utility(attributes,
                                  self.base_input_folder,
                                  output_folder,
                                  self.analysis_settings_config)
        # Return the initialized sample
        return(sample)

    
    ########## COPY OF ANOTHER METHOD TO RECHECK #############
    def write_to_tsv(self,
                     include_metadata = True):
        """
        Write results from all analyses to .tsv file. Includes metadata by default.

        Parameters:
        -------------

        include_metadata : bool
            Wether to include metadata
        """       
        
        if self.samples:
            output_file = os.path.join(self.base_output_folder,
                                       "results_summary.tsv")
            print_dicts = []
            for sample in self.samples:
                if include_metadata:
                    print_dict = sample.metadata.copy()
                    if 'Illumina_read_files' in print_dict and isinstance(print_dict['Illumina_read_files'], list):
                        print_dict["Illumina_read_files"] = ",".join(print_dict["Illumina_read_files"])
                    print_dict = {'sample_name': print_dict.pop('sample_name'), **print_dict} # Move sample_name to first column before printing for readability
                else:
                    print_dict = {"sample_name": sample.sample_name}
                for analysis_alias in sample.analysis_results:
                    print_dict.update(sample.analysis_results[analysis_alias])
                print_dicts.append(print_dict.copy())
            try:
                with open(output_file, 'w') as o:
                    dict_writer = csv.DictWriter(o, print_dicts[0].keys(), delimiter='\t')
                    dict_writer.writeheader()
                    dict_writer.writerows(print_dicts)  
                o.close()     
                print(f"Analysis summary written to {output_file}")
            except Exception as e:
                print(f"ERROR while writing analysis summary to {output_file}: {e}")


########## COPY OF ANOTHER METHOD TO RECHECK #############
    def setup_log_config(self,
                         log_file,
                         log_level = "INFO"):
        logger = logging.getLogger()
        logging.basicConfig(
            level=log_level,
            #filename=str(log_file),
            encoding="utf-8",
            filemode="w",
            format="{asctime} - {levelname} - {message}",
            style="{",
            datefmt="%Y-%m-%d %H:%M:%S",
        )
        return(logger)


In [ ]:
# |export

def get_names_from_fasta(fasta_file):
    """
    Parses a FASTA file and returns list with headers 
    matching what would appear as qseqid or sseqid in blast

    Parameters:
    -------------
        fasta_file : str
            Path to the FASTA file.

    Returns:
    -------------
       sequence_names: dict
            Dictionary with sequence headers as keys.
    """
    with open(fasta_file) as f:
        sequence_names = {}
        for line in f:
            if line[0] == ">":
                line = line.rstrip("\n")[1:]
                sequence_names[line.split()[0]] = line
    return(sequence_names)

def print_analysis_options(analysis_config):
    """
    Function to print available analyses based on the configuration
   
    Parameters:
    -------------
    analysis_config: dict
            Configuration settings for the analysis, including analysis type and parameters.
    """
    print("The following analyses are available:\n")
    analyses_to_run = analysis_config["analyses_to_run"]
    
    # If analyses are defined as a dictionary, print each with its description and output files
    if isinstance(analyses_to_run,dict):
        for analysis,description in analyses_to_run.items():
            print(f"{analysis}: {description}")
            try:
                # Print output files to be cleaned after the analysis
                output_files_print = "\n    - ".join(analysis_config[analysis]["files_to_clean"])
                print(f" output_files:\n    - {output_files_print}\n")
            except:
                print("\n")
    
    # If analyses are a list, print each analysis name
    elif isinstance(analyses_to_run,list):
        for analysis in analyses_to_run:
            print(f"{analysis}: {description}")
            try:
                output_files_print = "\n    - ".join(analysis_config[analysis]["files_to_clean"])
                print(f" output_files:\n    - {output_files_print}\n")
            except:
                print("\n")
    else:
        print("No analyses found in provided config")


def update_cleanup_config(analysis_config,
                          files_to_keep_string):
    
    """
    Function to update the cleanup configuration to keep specified files
    
    Parameters:
    -------------
    analysis_config: dict
            Configuration settings for the analysis, including analysis type and parameters.
    files_to_keep_string: str
            Files to keep
    """
    files_to_keep_list = files_to_keep_string.split(' ')
    for element in files_to_keep_list:
        element_split = element.split(':')
        if element_split[0] in analysis_config:
            if len(element_split) == 1:
                analysis_config[element]["files_to_clean"] = []
            else:
                analysis_config[element_split[0]]["files_to_clean"] = list(set(analysis_config[element_split[0]]["files_to_clean"])-set(element_split[1].split(',')))
    return(analysis_config)

def update_cli_input_config(input_config,
                            analysis_config,
                            samplesheet,
                            input_folder,
                            assembly_file,
                            Illumina_read_files,
                            Nanopore_read_file,
                            output_folder,
                            load_from_samplesheet,
                            analyses_to_run,
                            keep_files):
    
    """
    Updates input and analysis configuration dictionaries based on provided command-line arguments.
    
    Parameters:
    -------------
        input_config (dict): 
                The main configuration dictionary for input parameters.
        analysis_config (dict): 
                The configuration dictionary for analysis settings.
        samplesheet (str or None): 
                Path to the samplesheet file.
        input_folder (str or None): 
                Path to the folder containing input data.
        assembly_file (str or None): 
                Path to the assembly file.
        Illumina_read_files (list or None): 
                List of Illumina read file paths.
        Nanopore_read_file (str or None): 
                Path to the Nanopore read file.
        output_folder (str or None): 
                Path to the output folder.
        load_from_samplesheet (bool): 
                Whether to load configuration directly from the samplesheet.
        analyses_to_run (str): 
                Comma-separated list of analyses to run, or "all" to run all analyses.
        keep_files (str or None): 
                Specifies files to retain after cleanup.
        
    Returns:
    -------------
        tuple: Updated input_config and analysis_config dictionaries.
    """

    # Update the samplesheet if it was provided
    if samplesheet is not None:
        input_config["samplesheet"] = samplesheet
        # If no input folder is specified, load data from the samplesheet
        if input_folder is None:
            load_from_samplesheet = True
    
    # Update additional input files and folders in input_config if they are provided
    # Set the assembly file if it was specified
    if assembly_file is not None:
        input_config["assembly_file"] = assembly_file
    
    # Set the Illumina read files if they were specified
    if Illumina_read_files is not None:
        input_config["Illumina_read_files"] = Illumina_read_files
    
    # Set the Nanopore read file if it was specified
    if Nanopore_read_file is not None:
        input_config["Nanopore_read_file"] = Nanopore_read_file
    
    # Set the input folder if it was specified
    if input_folder is not None:
        input_config["input_folder"] = input_folder
        # Enable loading from the specified folder if not loading from sampleshee
        if not input_config["load_from_samplesheet"]:
            input_config["load_from_folder"] = True
    
    # Set the output folder if it was specified
    if output_folder is not None:
        input_config["output_folder"] = output_folder
    
    # Update analysis_config to specify which analyses to run
    # If `analyses_to_run` is not set to "all", update it with the intersection of provided analyses and existing analyses
    if not analyses_to_run == "all":
        analysis_config["analyses_to_run"] = list(set(analysis_config["analyses_to_run"]).intersection(analyses_to_run.split(',')))
    
    # Update files to keep based on `keep_files` argument
    if keep_files is not None:
        analysis_config = update_cleanup_config(analysis_config,keep_files)
    input_config["load_from_samplesheet"] = load_from_samplesheet
    
    # Return the updated configuration dictionaries
    return(input_config,
           analysis_config)

### Testing

The log parser unit test parses an rclone log found in tests/rclone_log_parser_test_log.txt. It verifies basic information like number of succesful and unsuccesful transfers in the log, and prints a tsv-formatted summary table of each transfer instance.

Should there be some try/except statements here, so the user get some kind of direct feedback like "all tests passed" (or not, if thats the case)? Right now, I dont think its very obvious whether the test was successful or not, from the user-side.

In [ ]:
# |export


def unit_test_single():
    config = core.get_config()
    example_sample = analysis_utility({"sample_name":"GAS-2022-1029",
                                       "assembly_file":"examples/GAS-2022-1029.fasta",
                                       "Illumina_read_files":["examples/GAS-2022-1029_S42_L555_R1_001.fastq.gz","examples/GAS-2022-1029_S42_L555_R2_001.fastq.gz"]},
                                       input_folder = False,
                                       output_folder = "output/",
                                       analysis_config = config["analysis_settings"]["Spyogenes"],
                                       )
    assert(example_sample.sample_name == "GAS-2022-1029")
    assert(len(example_sample.Illumina_read_files) == 2)
    assert(not example_sample.Nanopore_read_file)

def unit_test_single_2():
    config = core.get_config()
    example_sample = analysis_utility({"sample_name":"GAS-2024-0773",
                                       "assembly_file":"GAS-2024-0773.fasta",
                                       "Illumina_read_files":["GAS-2024-0773_S35_L555_R1_001.fastq.gz","GAS-2024-0773_S35_L555_R2_001.fastq.gz"]},
                                       input_folder = "examples",
                                       output_folder = "output/",
                                       analysis_config = config["analysis_settings"]["Spyogenes"],
                                       )
    assert(example_sample.sample_name == "GAS-2024-0773")
    assert(len(example_sample.Illumina_read_files) == 2)
    assert(not example_sample.Nanopore_read_file)

def unit_test_single_3():
    config = core.get_config()
    example_sample = analysis_utility({"sample_name":"GAS-2022-1029",
                                       "assembly_file":"examples/GAS-2022-1029.fasta",
                                       "Illumina_read_files":["examples/GAS-2022-1029_S42_L555_R1_001.fastq.gz","examples/GAS-2022-1029_S42_L555_R2_001.fastq.gz"],
                                       "samplesheet": "examples/samplesheet.tsv"},
                                       input_folder = False,
                                       output_folder = "output/",
                                       analysis_config = config["analysis_settings"]["Spyogenes"]
                                       )
    assert(example_sample.sample_name == "GAS-2022-1029")
    assert(len(example_sample.Illumina_read_files) == 2)
    assert(not example_sample.Nanopore_read_file)
    print(example_sample.metadata)


def unit_test_from_folder():
    config = core.get_config()
    input_config =  config["input_manager"]
    input_config["load_from_folder"] = True
    input_config["input_folder"] = "examples/"
    input_config["output_folder"] = "output_from_folder/"
    input_config["analysis_config"] = config["analysis_settings"]["Spyogenes"]
    test = analysis_manager(input_config,config["analysis_settings"]["Spyogenes"])

def unit_test_from_samplesheet():
    config = core.get_config()
    input_config =  config["input_manager"]
    input_config["load_from_samplesheet"] = True
    input_config["samplesheet"] = "examples/samplesheet.tsv"
    input_config["output_folder"] = "output_from_samplesheet/"
    input_config["analysis_config"] = config["analysis_settings"]["Spyogenes"]
    test = analysis_manager(input_config,config["analysis_settings"]["Spyogenes"])
    print(test.__dict__)
    for x in test:
        print(x.__dict__)



In [ ]:
# unit_test_single()

In [ ]:
# unit_test_single_2()

In [ ]:
# unit_test_single_3()

{'sample_name': 'GAS-2022-1029', 'assembly_file': 'examples/GAS-2022-1029.fasta', 'Illumina_read_files': ['examples/GAS-2022-1029_S42_L555_R1_001.fastq.gz', 'examples/GAS-2022-1029_S42_L555_R2_001.fastq.gz'], 'samplesheet': 'examples/samplesheet.tsv'}


In [ ]:

# unit_test_from_folder()

In [ ]:

# unit_test_from_samplesheet()

{'analysis_settings_config': {'analyses_to_run': {'emm_typing': 'Typing of M protein gene using blast against CDC curated alleles', 'assembly_lineage_determination': 'Lineage determination based on presence of specified SNPs when mapping genome assembly against reference genome', 'resistance_gene_detection': 'Presence of resistance gens tetM, ermA and ermB', 'virulence_gene_detection': 'Presence of Streptococcal virulence genes from the Virulence Factor Database'}, 'assembly_lineage_determination': {'alias': 'assembly_lineage_determination', 'percent_snp_threshold': 50, 'files_to_clean': ['delta', 'filtered_delta', 'frankenfasta'], 'reference_fasta_file': './resources/lineage_determination/MGAS5005.fasta', 'lineage_variant_file': './resources/lineage_determination/Spyogenes_LOCs.tsv'}, 'emm_typing': {'alias': 'emm_typing', 'emm_allele_file': './resources/emm_typing/emm_alleles.fasta', 'emm_cluster_file': './resources/emm_typing/emm_enn_mrp_subgroups.txt', 'blast_header': 'qseqid sseqid

In [ ]:
# config = core.get_config()

# print_analysis_options(config["analysis_settings"]["Spyogenes"])


The following analyses are available:

emm_typing: Typing of M protein gene using blast against CDC curated alleles
 output_files:
    - blast

assembly_lineage_determination: Lineage determination based on presence of specified SNPs when mapping genome assembly against reference genome
 output_files:
    - delta
    - filtered_delta
    - frankenfasta

resistance_gene_detection: Presence of resistance gens tetM, ermA and ermB
 output_files:
    - blast

virulence_gene_detection: Presence of Streptococcal virulence genes from the Virulence Factor Database
 output_files:
    - blast



#### Cli function

This function is responsible for the execution of the current notebook, including parsing arguments.

The current version of the code require specification of an input log-file. It can be specified using the --input argument, or by providing a config-file as argument (with the log-file specified). If neither argument is specified, the script will exit with an error.

It is also possible to run a "unit-test", by specyfying the "unit_test" flag. In that case, the script will run on the test log-file provided with the repo, and exit without doing anything else.

In [ ]:
# | hide
# This is included at the end to ensure when you run through your notebook the code is also transferred to the module and isn't just a notebook
import nbdev

nbdev.nbdev_export()